In [29]:
import numpy as np
import pandas as pd
# from skimage import imread
from matplotlib.image import imread
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
from sklearn.metrics import confusion_matrix, recall_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks
import sys
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle

import utils as ut

In [43]:
df_train_pred = pd.read_csv('./data/train/train_set.csv')
df_test_pred = pd.read_csv('./data/test/test_set.csv')
ROOT_PATH = os.getcwd()
TRAIN_PATH = "./data/img/train/"
TEST_PATH = "./data/img/test/"
feel_names = ['angry','disgust','fear','happy','neutral','sad','surprise']
feel_names_dict = {feel_name:i for i, feel_name in enumerate(feel_names)}

In [ ]:
### New: only in gray scale. Should be more efficient, again just for the first time
# x_train, y_train = ut.train_image_reader(TRAIN_PATH,'./data//processed/train/x1.pkl','./data//processed/train/y1.pkl')
# x_test = ut.test_image_reader(TEST_PATH,'./data/processed/test/x1.pkl')

Opening image #0 ,angry
Opening image #1 ,angry
Opening image #2 ,angry
Opening image #3 ,angry
Opening image #4 ,angry
Opening image #5 ,angry
Opening image #6 ,angry
Opening image #7 ,angry
Opening image #8 ,angry
Opening image #9 ,angry
Opening image #10 ,angry
Opening image #11 ,angry
Opening image #12 ,angry
Opening image #13 ,angry
Opening image #14 ,angry
Opening image #15 ,angry
Opening image #16 ,angry
Opening image #17 ,angry
Opening image #18 ,angry
Opening image #19 ,angry
Opening image #20 ,angry
Opening image #21 ,angry
Opening image #22 ,angry
Opening image #23 ,angry
Opening image #24 ,angry
Opening image #25 ,angry
Opening image #26 ,angry
Opening image #27 ,angry
Opening image #28 ,angry
Opening image #29 ,angry
Opening image #30 ,angry
Opening image #31 ,angry
Opening image #32 ,angry
Opening image #33 ,angry
Opening image #34 ,angry
Opening image #35 ,angry
Opening image #36 ,angry
Opening image #37 ,angry
Opening image #38 ,angry
Opening image #39 ,angry
Opening im

In [30]:
x_train = pickle.load(open('./data/processed/train/x1.pkl','rb'))
y_train = pickle.load(open('./data/processed/train/y1.pkl','rb'))
x_test = pickle.load(open('./data/processed/test/x1.pkl','rb'))

In [31]:
x_train.shape

(28821, 48, 48)

## SMOTE a las imágenes

In [32]:
x_train = x_train.reshape(x_train.shape[0], -1)
smote = SMOTE(random_state=42)
x_train,y_train = smote.fit_resample(x_train,y_train)
x_train = x_train.reshape(-1,48,48,1)
x_train.shape

(50148, 48, 48, 1)

In [25]:
np.unique(y_train,return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]),
 array([7164, 7164, 7164, 7164, 7164, 7164, 7164]))

In [33]:
datagen = ImageDataGenerator(
    rotation_range=20,  # Rotación aleatoria entre -20 y 20 grados
    width_shift_range=0.2,  # Desplazamiento horizontal aleatorio
    height_shift_range=0.2,  # Desplazamiento vertical aleatorio
    shear_range=0.2,  # Cortes aleatorios
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Volteo horizontal
    fill_mode='nearest'  # Rellenar píxeles vacíos con el valor más cercano
)

In [34]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=42)

In [35]:
model = models.Sequential()
model.add(layers.Conv2D(48, (3, 3), activation='relu', input_shape=(48, 48,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(96, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())

# Capa densa con Dropout
model.add(layers.Dense(48, activation='relu'))
# model.add(layers.Dropout(0.8))  # Dropout para reducir overfitting

model.add(layers.Dense(7, activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
### RUNTIME: 28 minutes
history = model.fit(datagen.flow(x_train,y_train),epochs=50,batch_size = 64,validation_data=(x_val,y_val),
          callbacks=callbacks.EarlyStopping(patience=5)
          )

Epoch 1/50


c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1411/1411 ━━━━━━━━━━━━━━━━━━━━ 38s 26ms/step - accuracy: 0.2016 - loss: 1.9054 - val_accuracy: 0.3061 - val_loss: 1.7619
Epoch 2/50
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 36s 26ms/step - accuracy: 0.2790 - loss: 1.8067 - val_accuracy: 0.3717 - val_loss: 1.6245
Epoch 3/50
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 40s 28ms/step - accuracy: 0.3282 - loss: 1.7238 - val_accuracy: 0.3996 - val_loss: 1.5583
Epoch 4/50
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 37s 26ms/step - accuracy: 0.3535 - loss: 1.6644 - val_accuracy: 0.4217 - val_loss: 1.5018
Epoch 5/50
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 37s 27ms/step - accuracy: 0.3739 - loss: 1.6192 - val_accuracy: 0.4327 - val_loss: 1.4974
Epoch 6/50
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 37s 26ms/step - accuracy: 0.3919 - loss: 1.5797 - val_accuracy: 0.4572 - val_loss: 1.4294
Epoch 7/50
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 37s 26ms/step - accuracy: 0.3960 - loss: 1.5679 - val_accuracy: 0.4830 - val_loss: 1.3832
Epoch 8/50
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 37s 26ms/step - accuracy: 0.4131 - loss: 1.53

## Prediction and results to csv

In [40]:
model.save("./models/model_2.keras")
# model = models.load_model("./models/model_2.keras")

In [41]:
y_pred = model.predict(x_test).argmax(axis=1)

221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [45]:
y_pred_names = [feel_names[x] for x in y_pred]
y_pred_names[:5]

['angry', 'surprise', 'angry', 'surprise', 'sad']

In [46]:
df_test_pred['label'] = y_pred_names
df_test_pred.head(3)

,id_img,label
0,10052,angry
1,10065,surprise
2,10079,angry


In [47]:
df_test_pred.to_csv('./data/results/results_2.csv',index=False)